<a href="https://colab.research.google.com/github/CoreTheGreat/HBPU-Machine-Learning-Course/blob/main/ML_Chapter3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 实验八：基于全连接深度神经网络模型的手写识别及Wi-Fi动作感知
湖北理工学院《机器学习》课程NoteBook

学生：吴兴平

笔记内容概述: 前馈神经网络、全连接网络、手写字母识别

### 按照Classification章节，读取MNIST数据集

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

label_size = 18 # Label size
ticklabel_size = 14 # Tick label size

class FlattenTransform:
    def __call__(self, tensor):
        ''' 
        Flatten tensor into an 1-D vector
        '''
        return tensor.view(-1)
    
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    FlattenTransform()
])

# Load test data from the MNIST
testset = torchvision.datasets.MNIST(root='./Data', train=False, download=False, transform=transform)
print(f"Test set size: {len(testset)}")

# Load training data from the MNIST
trainset = torchvision.datasets.MNIST(root='./Data', train=True, download=False, transform=transform)
print(f"Training set size: {len(trainset)}")

# Rate of trX and cvX
tr_cv_rate = 0.8

# Create a list to store indices for each class
class_indices = [[] for _ in range(10)]  # 10 classes in MNIST

# Populate class_indices
for idx, (_, label) in enumerate(trainset):
    class_indices[label].append(idx)

# Calculate the number of samples for each class in training and validation sets
train_size_per_class = int(tr_cv_rate * min(len(indices) for indices in class_indices))
val_size_per_class = min(len(indices) for indices in class_indices) - train_size_per_class

# Create balanced train and validation sets
train_indices = []
val_indices = []
for indices in class_indices:
    train_indices.extend(indices[:train_size_per_class])
    val_indices.extend(indices[train_size_per_class:train_size_per_class + val_size_per_class])

# Create Subset datasets
from torch.utils.data import Subset
trX = Subset(trainset, train_indices)
cvX = Subset(trainset, val_indices)

print(f"Number of training samples: {len(trX)}")
print(f"Number of cross-validation samples: {len(cvX)}")

Test set size: 10000
Training set size: 60000
Number of training samples: 43360
Number of cross-validation samples: 10850


构建DataLoaders，准备训练模型

In [4]:
batch_size = 42 # Define training batch

def one_hot_collate(batch):
    data = torch.stack([item[0] for item in batch])
    labels = torch.tensor([item[1] for item in batch])
    one_hot_labels = torch.zeros(labels.size(0), 10)  # 10 classes in MNIST
    one_hot_labels.scatter_(1, labels.unsqueeze(1), 1)
    return data, one_hot_labels

trLoader = torch.utils.data.DataLoader(trX, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=one_hot_collate)
cvLoader = torch.utils.data.DataLoader(cvX, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=one_hot_collate)
teLoader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=one_hot_collate)

# Get a batch of training data
dataiter = iter(trLoader)
data, labels = next(dataiter)

input_size = data[0].numpy().shape[0]
print(f'Input_size is {input_size}')
print(labels)

Input_size is 784
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.,

### 定义并训练全连接神经网络
* 输入：1-D向量
* 输出：手写字母类型的概率分布
* 隐藏层：2层
* 节点数：100个/层

In [6]:
import torch.nn as nn

class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        out = self.softmax(x)
        return out

# Define the model parameters
hidden_size = 10
num_classes = 10  # MNIST has 10 classes (digits 0-9)

# Instantiate the model
model = FNN(input_size, hidden_size, num_classes)
print(model)

FNN(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


使用Adam作为Optimizor训练模型

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Lists to store losses
train_losses = []
cv_losses = []

# Number of epochs
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    batch_losses = []
    
    for batch_x, batch_y in trLoader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_losses.append(loss.item())
    
    # Calculate average training loss for this epoch
    avg_train_loss = sum(batch_losses) / len(batch_losses)
    train_losses.append(avg_train_loss)
    
    # Evaluate on cross-validation set
    model.eval()
    cv_batch_losses = []
    with torch.no_grad():
        for cv_x, cv_y in cvLoader:
            cv_outputs = model(cv_x)
            cv_loss = criterion(cv_outputs, cv_y)
            cv_batch_losses.append(cv_loss.item())
    
    avg_cv_loss = sum(cv_batch_losses) / len(cv_batch_losses)
    cv_losses.append(avg_cv_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, CV Loss: {avg_cv_loss:.4f}')

Epoch [1/50], Train Loss: 1.7977, CV Loss: 1.6700
Epoch [2/50], Train Loss: 1.6532, CV Loss: 1.6492
Epoch [3/50], Train Loss: 1.6394, CV Loss: 1.6422
Epoch [4/50], Train Loss: 1.6329, CV Loss: 1.6385
Epoch [5/50], Train Loss: 1.6281, CV Loss: 1.6352
Epoch [6/50], Train Loss: 1.6255, CV Loss: 1.6393
Epoch [7/50], Train Loss: 1.6231, CV Loss: 1.6351
Epoch [8/50], Train Loss: 1.6204, CV Loss: 1.6335
Epoch [9/50], Train Loss: 1.6189, CV Loss: 1.6294
Epoch [10/50], Train Loss: 1.6172, CV Loss: 1.6268
Epoch [11/50], Train Loss: 1.6159, CV Loss: 1.6280
Epoch [12/50], Train Loss: 1.6151, CV Loss: 1.6271
Epoch [13/50], Train Loss: 1.6138, CV Loss: 1.6260
Epoch [14/50], Train Loss: 1.6121, CV Loss: 1.6248
Epoch [15/50], Train Loss: 1.6115, CV Loss: 1.6245
Epoch [16/50], Train Loss: 1.6107, CV Loss: 1.6245
Epoch [17/50], Train Loss: 1.6097, CV Loss: 1.6261
Epoch [18/50], Train Loss: 1.6086, CV Loss: 1.6239
Epoch [19/50], Train Loss: 1.6083, CV Loss: 1.6217
Epoch [20/50], Train Loss: 1.6072, CV Lo

计算识别精度，展示学习曲线

In [ ]:
# Calculate and print accuracies for training and cross-validation sets
model.eval()
with torch.no_grad():
    # Training set accuracy
    tr_correct = 0
    tr_total = 0
    for images, labels in trLoader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        _, true_labels = torch.max(labels, 1)
        tr_total += labels.size(0)
        tr_correct += (predicted == true_labels).sum().item()
    
    tr_accuracy = 100 * tr_correct / tr_total
    
    # Cross-validation set accuracy
    cv_correct = 0
    cv_total = 0
    for images, labels in cvLoader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        _, true_labels = torch.max(labels, 1)
        cv_total += labels.size(0)
        cv_correct += (predicted == true_labels).sum().item()
    
    cv_accuracy = 100 * cv_correct / cv_total

print(f'Accuracy on training set: {tr_accuracy:.2f}%')
print(f'Accuracy on cross-validation set: {cv_accuracy:.2f}%')

# Plot training and cross-validation losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), cv_losses, label='Cross-Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Cross-Validation Loss')
plt.legend()
plt.show()

Accuracy on training set: 86.91%
Accuracy on cross-validation set: 84.42%
